In [1]:
import sqlite3

# Conectar a la base de datos
db_path = "unique_db_inmuebles.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Habilitar claves foráneas
cursor.execute("PRAGMA foreign_keys = ON;")
print("Claves foráneas habilitadas.")

# Eliminar y recrear la tabla inmuebles_combined con relaciones correctas
cursor.execute("DROP TABLE IF EXISTS inmuebles_combined;")
cursor.execute("""
CREATE TABLE inmuebles_combined (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo_operacion_origen TEXT,
    descripcion TEXT,
    localizacion TEXT,
    precio REAL,
    ultima_actualizacion TEXT,
    tipo_operacion TEXT,
    superficie_construida REAL,
    superficie_util TEXT,
    habitaciones INTEGER,
    baños INTEGER,
    antigüedad TEXT,
    conservacion TEXT,
    cp INTEGER,
    planta TEXT,
    tipo_casa TEXT,
    -- Relación con alquiler_data
    FOREIGN KEY (localizacion, precio, superficie_construida, tipo_casa, cp)
    REFERENCES alquiler_data (localizacion, precio, superficie_construida, tipo_casa, cp)
    ON DELETE CASCADE ON UPDATE CASCADE,
    -- Relación con venta_data
    FOREIGN KEY (localizacion, precio, superficie_construida, tipo_casa, cp)
    REFERENCES venta_data (localizacion, precio, superficie_construida, tipo_casa, cp)
    ON DELETE CASCADE ON UPDATE CASCADE
);
""")
print("Tabla inmuebles_combined creada correctamente.")

# Intentar insertar datos desde alquiler_data
try:
    cursor.execute("""
    INSERT INTO inmuebles_combined (tipo_operacion_origen, descripcion, localizacion, precio, ultima_actualizacion, 
        tipo_operacion, superficie_construida, superficie_util, habitaciones, baños, antigüedad, conservacion, 
        cp, planta, tipo_casa)
    SELECT 
        'alquiler', descripcion, localizacion, precio, ultima_actualizacion, 
        tipo_operacion, superficie_construida, superficie_util, habitaciones, baños, antigüedad, conservacion, 
        cp, planta, tipo_casa
    FROM alquiler_data;
    """)
    print("Datos insertados desde alquiler_data correctamente.")
except sqlite3.OperationalError as e:
    print(f"Error insertando desde alquiler_data: {e}")

# Intentar insertar datos desde venta_data
try:
    cursor.execute("""
    INSERT INTO inmuebles_combined (tipo_operacion_origen, descripcion, localizacion, precio, ultima_actualizacion, 
        tipo_operacion, superficie_construida, superficie_util, habitaciones, baños, antigüedad, conservacion, 
        cp, planta, tipo_casa)
    SELECT 
        'venta', descripcion, localizacion, precio, ultima_actualizacion, 
        tipo_operacion, superficie_construida, superficie_util, habitaciones, baños, antigüedad, conservacion, 
        cp, planta, tipo_casa
    FROM venta_data;
    """)
    print("Datos insertados desde venta_data correctamente.")
except sqlite3.OperationalError as e:
    print(f"Error insertando desde venta_data: {e}")

# Verificar claves foráneas configuradas
cursor.execute("PRAGMA foreign_key_list('inmuebles_combined');")
foreign_keys = cursor.fetchall()
print("\nClaves foráneas configuradas en inmuebles_combined:")
for fk in foreign_keys:
    print(fk)

# Confirmar y cerrar conexión
conn.commit()
conn.close()
print("Proceso completado correctamente.")


Claves foráneas habilitadas.
Tabla inmuebles_combined creada correctamente.
Error insertando desde alquiler_data: foreign key mismatch - "inmuebles_combined" referencing "venta_data"
Error insertando desde venta_data: foreign key mismatch - "inmuebles_combined" referencing "venta_data"

Claves foráneas configuradas en inmuebles_combined:
(0, 0, 'venta_data', 'localizacion', 'localizacion', 'CASCADE', 'CASCADE', 'NONE')
(0, 1, 'venta_data', 'precio', 'precio', 'CASCADE', 'CASCADE', 'NONE')
(0, 2, 'venta_data', 'superficie_construida', 'superficie_construida', 'CASCADE', 'CASCADE', 'NONE')
(0, 3, 'venta_data', 'tipo_casa', 'tipo_casa', 'CASCADE', 'CASCADE', 'NONE')
(0, 4, 'venta_data', 'cp', 'cp', 'CASCADE', 'CASCADE', 'NONE')
(1, 0, 'alquiler_data', 'localizacion', 'localizacion', 'CASCADE', 'CASCADE', 'NONE')
(1, 1, 'alquiler_data', 'precio', 'precio', 'CASCADE', 'CASCADE', 'NONE')
(1, 2, 'alquiler_data', 'superficie_construida', 'superficie_construida', 'CASCADE', 'CASCADE', 'NONE')
(

In [2]:
import sqlite3

# Conexión a la base de datos
db_path = "unique_db_inmuebles.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Verificar columnas en `alquiler_data`
cursor.execute("PRAGMA table_info(alquiler_data);")
columnas_alquiler = [col[1] for col in cursor.fetchall()]
print("Columnas en alquiler_data:", columnas_alquiler)

# Si 'cp' no existe en `alquiler_data`, agregarla
if "cp" not in columnas_alquiler:
    cursor.execute("ALTER TABLE alquiler_data ADD COLUMN cp INTEGER;")
    print("Columna 'cp' agregada a la tabla alquiler_data.")

# Verificar columnas en `venta_data`
cursor.execute("PRAGMA table_info(venta_data);")
columnas_venta = [col[1] for col in cursor.fetchall()]
print("Columnas en venta_data:", columnas_venta)

# Si 'cp' no existe en `venta_data`, agregarla
if "cp" not in columnas_venta:
    cursor.execute("ALTER TABLE venta_data ADD COLUMN cp INTEGER;")
    print("Columna 'cp' agregada a la tabla venta_data.")

# Confirmar cambios
conn.commit()
conn.close()
print("Verificación y aseguramiento de columnas completados.")


Columnas en alquiler_data: ['id', 'descripcion', 'localizacion', 'precio', 'ultima_actualizacion', 'tipo_operacion', 'superficie_construida', 'superficie_util', 'habitaciones', 'baños', 'antigüedad', 'conservacion', 'codigo_postal', 'planta', 'tipo_casa', 'cp']
Columnas en venta_data: ['id', 'descripcion', 'localizacion', 'precio', 'ultima_actualizacion', 'tipo_operacion', 'superficie_construida', 'superficie_util', 'habitaciones', 'baños', 'antigüedad', 'conservacion', 'cp', 'planta', 'tipo_casa']
Verificación y aseguramiento de columnas completados.


In [3]:
import sqlite3

# Conexión a la base de datos
db_path = "unique_db_inmuebles.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Insertar datos en `inmuebles_combined` desde `alquiler_data`
try:
    cursor.execute("""
    INSERT INTO inmuebles_combined (tipo_operacion_origen, descripcion, localizacion, precio, ultima_actualizacion, 
        tipo_operacion, superficie_construida, superficie_util, habitaciones, baños, antigüedad, conservacion, 
        cp, planta, tipo_casa)
    SELECT 
        'alquiler', descripcion, localizacion, precio, ultima_actualizacion, 
        tipo_operacion, superficie_construida, superficie_util, habitaciones, baños, antigüedad, conservacion, 
        cp, planta, tipo_casa
    FROM alquiler_data;
    """)
    print("Datos insertados desde alquiler_data correctamente.")
except sqlite3.OperationalError as e:
    print(f"Error insertando desde alquiler_data: {e}")

# Insertar datos en `inmuebles_combined` desde `venta_data`
try:
    cursor.execute("""
    INSERT INTO inmuebles_combined (tipo_operacion_origen, descripcion, localizacion, precio, ultima_actualizacion, 
        tipo_operacion, superficie_construida, superficie_util, habitaciones, baños, antigüedad, conservacion, 
        cp, planta, tipo_casa)
    SELECT 
        'venta', descripcion, localizacion, precio, ultima_actualizacion, 
        tipo_operacion, superficie_construida, superficie_util, habitaciones, baños, antigüedad, conservacion, 
        cp, planta, tipo_casa
    FROM venta_data;
    """)
    print("Datos insertados desde venta_data correctamente.")
except sqlite3.OperationalError as e:
    print(f"Error insertando desde venta_data: {e}")

# Confirmar cambios
conn.commit()
conn.close()
print("Datos insertados correctamente en inmuebles_combined.")


Datos insertados desde alquiler_data correctamente.
Datos insertados desde venta_data correctamente.
Datos insertados correctamente en inmuebles_combined.
